In [1]:
import os
import sys
import torchvision
import json
from transformers.tokenization_bert import BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/media/moju/data/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
import torch
import torch.nn as nn
import torchvision
from transformers.tokenization_bert import BertTokenizer
import config
from fgc_preprocess import FgcSerDataset, BertIdx
from torch.utils.data import DataLoader
from sup_model import BertSupSentClassification
from transformers import BertModel
from transformers.optimization import AdamW, WarmupLinearSchedule
from tqdm import tqdm_notebook as tqdm
from fgc_support_retri.fgc_preprocess import HotpotDataset

In [4]:
config.HOTPOT_DEV

PosixPath('/media/moju/data/work/fgc_support_retri/data/hotpot_dataset/FGC_hotpot_dev_distractor_v1(cn_refn).json')

In [5]:
def get_sup(item):
    item['SENTS'] = item['DTEXT'].split('\n')[:-1]
    sup_evidence = [0] * len(item['SENTS'])
    
    for shi in item['SHINT']:
        for s_i, s in enumerate(item['SENTS']):
            if shi['text'] == s:
                sup_evidence[s_i] = 1
    item['SUP_EVIDENCE'] = sup_evidence

In [6]:
with open(config.HOTPOT_DEV) as f:
    items = json.load(f)
print("{} questions".format(len(items)))

for item in items:
    assert len(item['QUESTIONS']) == 1
    for k, v in item['QUESTIONS'][0].items():
        item[k] = v
    del item['QUESTIONS']
    get_sup(item)
    assert 1 in item['SUP_EVIDENCE']
    del item['SHINT']
    del item ['ASPAN']

7405 questions


In [7]:
items[0]

{'DID': '5a8b57f25542995d1e6f1371',
 'DTEXT': '斯科特·德里克森（1966年7月16日出生）是美国导演、编剧和制片人。\n他住在加州洛杉矶。\n他最出名的是执导恐怖电影，如《险恶》、《艾米丽·罗斯的驱魔》、《让我们远离邪恶》，以及2016年惊奇电影宇宙版的《奇怪医生》\n爱德华·戴维斯·伍德（1924年10月10日至1978年12月10日）是美国电影制片人、演员、作家、制片人和导演。\n',
 'QID': '1',
 'QTEXT': '斯科特·德里克森和埃德·伍德是同一国籍的吗？',
 'QTYPE': '进阶题',
 'AMODE': 'Comparing-Members',
 'ANSWER': [{'ATEXT': '对', 'ATOKEN': {'text': '对', 'start': 0}},
  {'ATEXT': 'yes', 'ATOKEN': {'text': 'yes', 'start': 0}}],
 'SENTS': ['斯科特·德里克森（1966年7月16日出生）是美国导演、编剧和制片人。',
  '他住在加州洛杉矶。',
  '他最出名的是执导恐怖电影，如《险恶》、《艾米丽·罗斯的驱魔》、《让我们远离邪恶》，以及2016年惊奇电影宇宙版的《奇怪医生》',
  '爱德华·戴维斯·伍德（1924年10月10日至1978年12月10日）是美国电影制片人、演员、作家、制片人和导演。'],
 'SUP_EVIDENCE': [1, 0, 0, 1]}

In [8]:
bert_model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
dataset = HotpotDataset(items, transform=BertIdx(tokenizer))

In [9]:
dataset[0].keys()

dict_keys(['DID', 'SID', 'QTEXT', 'sentence', 'label', 'input_ids', 'token_type_ids'])

In [10]:
len(dataset[0]['input_ids'])

56

In [11]:
len(dataset[0]['token_type_ids'])

56

In [12]:
len(dataset)

47988

In [13]:
dataloader_train = DataLoader(dataset, batch_size=2)

In [15]:
for batch in dataloader_train:
    print(batch['input_ids'])
    import pdb; pdb.set_trace()

[tensor([101, 101]), tensor([3172, 3172]), tensor([4906, 4906]), tensor([4294, 4294]), tensor([185, 185]), tensor([2548, 2548]), tensor([7027, 7027]), tensor([1046, 1046]), tensor([3481, 3481]), tensor([1469, 1469]), tensor([1812, 1812]), tensor([2548, 2548]), tensor([185, 185]), tensor([824, 824]), tensor([2548, 2548]), tensor([3221, 3221]), tensor([1398, 1398]), tensor([671, 671]), tensor([1744, 1744]), tensor([5093, 5093]), tensor([4638, 4638]), tensor([1408, 1408]), tensor([8043, 8043]), tensor([102, 102]), tensor([3172,  800]), tensor([4906,  857]), tensor([4294, 1762]), tensor([ 185, 1217]), tensor([2548, 2336]), tensor([7027, 3821]), tensor([1046, 3329]), tensor([3481, 4768]), tensor([8020,  511]), tensor([9093,  102])]
> <ipython-input-15-3a221d6f0dc4>(1)<module>()
-> for batch in dataloader_train:


(Pdb)  exit


BdbQuit: 